In [1]:
!pip install open-unmix


ERROR: Could not find a version that satisfies the requirement open-unmix (from versions: none)
ERROR: No matching distribution found for open-unmix


In [2]:
import torch
import openunmix

# Load pre-trained model
separator = openunmix.umxl()

In [10]:
separator = torch.hub.load('sigsep/open-unmix-pytorch', 'umxl', device='cpu')  # Change 'cpu' to 'cuda' if you have a GPU.

Using cache found in C:\Users\anand/.cache\torch\hub\sigsep_open-unmix-pytorch_master


In [27]:
from pydub import AudioSegment

# Function to convert mp3 to wav
def convert_mp3_to_wav(mp3_file, wav_file):
    # Load the mp3 file
    audio = AudioSegment.from_mp3(mp3_file)
    
    # Export as wav
    audio.export(wav_file, format="wav")

# Example usage
mp3_file_path = r"D:\Tag\Vocal Gender Classifier\Test Songs\Megham Karukatha.mp3"  # Replace with your MP3 file path
wav_file_path = r"D:\Tag\Vocal Gender Classifier\Test Songs\Megham Karukatha.wav"   # Replace with your desired WAV file path

convert_mp3_to_wav(mp3_file_path, wav_file_path)

print(f"Converted {mp3_file_path} to {wav_file_path}")


Converted D:\Tag\Vocal Gender Classifier\Test Songs\Megham Karukatha.mp3 to D:\Tag\Vocal Gender Classifier\Test Songs\Megham Karukatha.wav


In [22]:
import openunmix
print("Open-Unmix imported successfully!")


Open-Unmix imported successfully!


In [31]:
import torch
import openunmix
import torchaudio

# Function to separate and save vocals and accompaniment from an audio file
def separate_audio(audio_path, output_vocals='vocals.wav', output_accompaniment='accompaniment.wav'):
    # Load the pre-trained model
    separator = openunmix.umxl()

    # Load the audio file and preprocess it
    try:
        audio, sample_rate = torchaudio.load(audio_path)
        print(f"Loaded audio shape: {audio.shape}, Sample rate: {sample_rate}")
    except Exception as e:
        print(f"Error loading audio file: {e}")
        return

    # Ensure audio has the correct shape (1, C, T)
    if audio.dim() == 1:  # mono
        audio = audio.unsqueeze(0).unsqueeze(0)  # Shape (1, 1, T)
    elif audio.dim() == 2:  # stereo
        audio = audio.unsqueeze(0)  # Shape (1, C, T)

    print(f"Audio shape after unsqueeze: {audio.shape}")

    # Perform separation
    with torch.no_grad():  # Disable gradient calculation
        estimates = separator(audio)

    # Check the shape of the estimates
    print(f"Estimates shape: {estimates.shape}")

    # Extract the vocal and accompaniment parts
    if estimates.dim() == 3:
        # Assuming estimates are in the shape [1, 2, C, T]
        vocals = estimates[0, 0]  # First output for vocals
        accompaniment = estimates[0, 1]  # Second output for accompaniment
    elif estimates.dim() == 4 and estimates.shape[1] >= 2:
        # If estimates are in the shape [1, 4, 2, T] (two sources per estimate)
        vocals = estimates[0, 0]  # First channel corresponds to vocals
        accompaniment = estimates[0, 1]  # Second channel corresponds to accompaniment
    else:
        print("Estimation did not return expected shape for vocals and accompaniment. Exiting.")
        return

    # Check and print the shape of the extracted vocals and accompaniment
    print(f"Extracted vocals shape: {vocals.shape}")
    print(f"Extracted accompaniment shape: {accompaniment.shape}")

    # Ensure we save 2D tensors
    if vocals.dim() == 3:  # If the vocals tensor is still 3D
        vocals = vocals.squeeze(0)  # Reduce to [C, T]

    if accompaniment.dim() == 3:  # If the accompaniment tensor is still 3D
        accompaniment = accompaniment.squeeze(0)  # Reduce to [C, T]

    # Save the vocal audio file
    try:
        torchaudio.save(output_vocals, vocals, sample_rate=sample_rate)  # Save the extracted vocals
        print(f"Separated vocals saved as '{output_vocals}'.")
    except Exception as e:
        print(f"Error saving vocal audio file: {e}")

    # Save the accompaniment audio file
    try:
        torchaudio.save(output_accompaniment, accompaniment, sample_rate=sample_rate)  # Save the extracted accompaniment
        print(f"Separated accompaniment saved as '{output_accompaniment}'.")
    except Exception as e:
        print(f"Error saving accompaniment audio file: {e}")

# Specify the path to your audio file
audio_file_path = r'D:\Tag\Vocal Gender Classifier\Test Songs\Megham Karukatha.wav'  # Replace with your audio file

# Call the separation function
separate_audio(audio_file_path)


Loaded audio shape: torch.Size([2, 12809216]), Sample rate: 44100
Audio shape after unsqueeze: torch.Size([1, 2, 12809216])
Estimates shape: torch.Size([1, 4, 2, 12809216])
Extracted vocals shape: torch.Size([2, 12809216])
Extracted accompaniment shape: torch.Size([2, 12809216])
Separated vocals saved as 'vocals.wav'.
Separated accompaniment saved as 'accompaniment.wav'.
